A* Algorithm for Pathfinding


In [ ]:
import heapq
def heuristic(a, b):
    # Using Manhattan Distance
    return abs(a[0] - b[0]) + abs(a[1] - b[1])
def a_star(grid):
    start = (0, 0)
    goal = (4, 4)
    open_set = []
    heapq.heappush(open_set, (0, start))
    came_from = {}
    g_score = {start: 0}
    f_score = {start: heuristic(start, goal)}
    while open_set:
        current = heapq.heappop(open_set)[1]
        if current == goal:
            return reconstruct_path(came_from, current)
        for neighbor in get_neighbors(current, grid):
            tentative_g_score = g_score[current] + 1
            if tentative_g_score < g_score.get(neighbor, float('inf')):
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = tentative_g_score + heuristic(neighbor, goal)
                if neighbor not in [i[1] for i in open_set]:
                    heapq.heappush(open_set, (f_score[neighbor], neighbor))

    return None  # No path found

def get_neighbors(position, grid):
    neighbors = []
    x, y = position
    for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        nx, ny = x + dx, y + dy
        if 0 <= nx < len(grid) and 0 <= ny < len(grid[0]) and grid[nx][ny] == 0:
            neighbors.append((nx, ny))
    return neighbors

def reconstruct_path(came_from, current):
    total_path = [current]
    while current in came_from:
        current = came_from[current]
        total_path.append(current)
    return total_path[::-1]

# Example grid (0 = free space, 1 = obstacle)
grid = [
    [0, 0, 0, 0, 1],
    [0, 1, 1, 0, 1],
    [0, 0, 0, 0, 0],
    [1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0]
]

path = a_star(grid)

# Mark the path on the grid
for x, y in path:
    if (x, y) != (0, 0) and (x, y) != (4, 4):
        grid[x][y] = 'P'

for row in grid:
    print(row)

[0, 'P', 'P', 'P', 1]
[0, 1, 1, 'P', 1]
[0, 0, 0, 'P', 'P']
[1, 1, 1, 1, 'P']
[0, 0, 0, 0, 0]


Water-Jug Problem Using A* Algorithm


In [4]:
from collections import deque

class WaterJugState:
    def __init__(self, jug1, jug2, target, path=[]):
        self.jug1 = jug1
        self.jug2 = jug2
        self.target = target
        self.path = path

    def is_goal(self):
        return self.jug1 == self.target or self.jug2 == self.target

    def get_possible_actions(self):
        actions = []
        # Fill Jug 1
        actions.append(WaterJugState(4, self.jug2, self.target, self.path + [(4, self.jug2)]))
        # Fill Jug 2
        actions.append(WaterJugState(self.jug1, 3, self.target, self.path + [(self.jug1, 3)]))
        # Empty Jug 1
        actions.append(WaterJugState(0, self.jug2, self.target, self.path + [(0, self.jug2)]))
        # Empty Jug 2
        actions.append(WaterJugState(self.jug1, 0, self.target, self.path + [(self.jug1, 0)]))
        # Pour Jug 1 to Jug 2
        pour_to_jug2 = min(self.jug1, 3 - self.jug2)
        actions.append(WaterJugState(self.jug1 - pour_to_jug2, self.jug2 + pour_to_jug2, self.target, self.path + [(self.jug1 - pour_to_jug2, self.jug2 + pour_to_jug2)]))
        # Pour Jug 2 to Jug 1
        pour_to_jug1 = min(self.jug2, 4 - self.jug1)
        actions.append(WaterJugState(self.jug1 + pour_to_jug1, self.jug2 - pour_to_jug1, self.target, self.path + [(self.jug1 + pour_to_jug1, self.jug2 - pour_to_jug1)]))
        return actions

def water_jug_a_star(initial_state):
    open_list = deque([initial_state])
    visited = set()

    while open_list:
        current_state = open_list.popleft()

        if current_state.is_goal():
            return current_state.path

        visited.add((current_state.jug1, current_state.jug2))
        for action in current_state.get_possible_actions():
            if (action.jug1, action.jug2) not in visited:
                open_list.append(action)

    return None  # No solution found

# Initial state (0, 0), target is to measure 2 liters
initial_state = WaterJugState(0, 0, 2)
solution = water_jug_a_star(initial_state)
if solution:
    print("Steps to achieve target:")
    for step in solution:
        print(step)
else:
    print("No solution found")



Steps to achieve target:
(0, 3)
(3, 0)
(3, 3)
(4, 2)


**Hill-Climbing Algorithm for the 8-Queen Problem**

In [5]:
import random

def print_board(queens):
    board = [['.'] * 8 for _ in range(8)]
    for col in range(8):
        board[queens[col]][col] = 'Q'
    for row in board:
        print(' '.join(row))
    print()

def calculate_heuristic(queens):
    conflicts = 0
    for i in range(len(queens)):
        for j in range(i + 1, len(queens)):
            if queens[i] == queens[j] or abs(queens[i] - queens[j]) == abs(i - j):
                conflicts += 1
    return conflicts

def hill_climbing():
    # Random initial state
    queens = list(range(8))
    random.shuffle(queens)

    while True:
        current_conflicts = calculate_heuristic(queens)
        if current_conflicts == 0:
            print("Solution found:")
            print_board(queens)
            return queens

        # Generate neighbors
        neighbors = []
        for col in range(8):
            for row in range(8):
                if row != queens[col]:
                    new_queens = queens[:]
                    new_queens[col] = row
                    neighbors.append(new_queens)

        # Select the best neighbor
        neighbors.sort(key=calculate_heuristic)
        best_neighbor = neighbors[0]
        best_conflicts = calculate_heuristic(best_neighbor)

        if best_conflicts >= current_conflicts:
            print("No solution found, stuck in local maximum.")
            return None

        queens = best_neighbor

hill_climbing()


Solution found:
. . Q . . . . .
. . . . . . Q .
. Q . . . . . .
. . . . . . . Q
. . . . . Q . .
. . . Q . . . .
Q . . . . . . .
. . . . Q . . .



[6, 2, 0, 5, 7, 4, 1, 3]

**Mini-Max Algorithm for Tic-Tac-Toe**

In [ ]:
import math

# Constants for players
PLAYER_X = "X"
PLAYER_O = "O"
EMPTY = " "

class TicTacToe:
    def __init__(self):
        self.board = [[EMPTY] * 3 for _ in range(3)]
        self.current_player = PLAYER_X

    def print_board(self):
        for row in self.board:
            print("|".join(row))
            print("-" * 5)

    def is_winner(self, player):
        # Check rows, columns and diagonals
        for i in range(3):
            if all([self.board[i][j] == player for j in range(3)]) or \
               all([self.board[j][i] == player for j in range(3)]):
                return True
        if all([self.board[i][i] == player for i in range(3)]) or \
           all([self.board[i][2 - i] == player for i in range(3)]):
            return True
        return False

    def is_draw(self):
        return all(cell != EMPTY for row in self.board for cell in row)

    def minimax(self, is_maximizing):
        if self.is_winner(PLAYER_X):
            return -1
        if self.is_winner(PLAYER_O):
            return 1
        if self.is_draw():
            return 0

        if is_maximizing:
            best_score = -math.inf
            for i in range(3):
                for j in range(3):
                    if self.board[i][j] == EMPTY:
                        self.board[i][j] = PLAYER_O
                        score = self.minimax(False)
                        self.board[i][j] = EMPTY
                        best_score = max(score, best_score)
            return best_score
        else:
            best_score = math.inf
            for i in range(3):
                for j in range(3):
                    if self.board[i][j] == EMPTY:
                        self.board[i][j] = PLAYER_X
                        score = self.minimax(True)
                        self.board[i][j] = EMPTY
                        best_score = min(score, best_score)
            return best_score

    def best_move(self):
        best_score = -math.inf
        move = (-1, -1)
        for i in range(3):
            for j in range(3):
                if self.board[i][j] == EMPTY:
                    self.board[i][j] = PLAYER_O
                    score = self.minimax(False)
                    self.board[i][j] = EMPTY
                    if score > best_score:
                        best_score = score
                        move = (i, j)
        return move

    def play(self):
        while True:
            self.print_board()
            if self.current_player == PLAYER_X:
                row, col = map(int, input("Enter your move (row and column): ").split())
                if self.board[row][col] == EMPTY:
                    self.board[row][col] = PLAYER_X
                    if self.is_winner(PLAYER_X):
                        self.print_board()
                        print("You win!")
                        break
                    self.current_player = PLAYER_O
            else:
                print("AI is making a move...")
                row, col = self.best_move()
                self.board[row][col] = PLAYER_O
                if self.is_winner(PLAYER_O):
                    self.print_board()
                    print("AI wins!")
                    break
                self.current_player = PLAYER_X

            if self.is_draw():
                self.print_board()
                print("It's a draw!")
                break

if __name__ == "__main__":
    game = TicTacToe()
    game.play()



 | | 
-----
 | | 
-----
 | | 
-----
Enter your move (row and column): 0 0
X| | 
-----
 | | 
-----
 | | 
-----
AI is making a move...
X| | 
-----
 |O| 
-----
 | | 
-----
Enter your move (row and column): 1 1
X| | 
-----
 |O| 
-----
 | | 
-----
